In [ ]:
# Import useful libraries
import json, pandas as pd, pydeck as pdk, ee, ipyfilechooser, ipywidgets, datetime, ee_satellites

In [ ]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()

# **Features Extraction**

Feature extraction is fundamental in Earth Observation (EO) because it enables us to extract valuable information from large and complex satellite data. EO is a field that involves the collection and analysis of data about the Earth's surface and atmosphere from satellites and other airborne sensors. The data acquired from these sensors are usually vast, complex, and contain a wealth of information that needs to be extracted to make sense of them.

## **Import a JSON file containing crop fields details**

The structure of the JSON file must comply with two very simple rules ([example](areas-of-interest.json)):
* it should contain the **crop field name** (preferred if each crop field have a unique name)
* it should contain the **set of coordinates** that are composing the single crop field (closed polygon geometry)

In [ ]:
# Choose the file (it must be a JSON file)
file_chooser = ipyfilechooser.FileChooser(path='./', filename='areas-of-interest.json', select_default=True, use_dir_icons=True, filter_pattern='*.json')
display(file_chooser)

In [ ]:
# Load JSON data from file
with open(file_chooser.selected) as f:
    data = json.load(f)

# Create DataFrame with properties excluding 'manure_dates' column
fields_df = pd.DataFrame([{k:v for k,v in f['properties'].items()} for f in data['features']])

# Add column with coordinates for each field
fields_df['polygon_coordinates'] = [[tuple(c) for c in p] for f in data['features'] for p in f['geometry']['coordinates']]

# Show the entire dataframe
display(fields_df)

## **Show crop fields locations on Earth-map**
The objective is to show where our crop fields are geographically placed. This can provide several benefits, including:
* **Spatial context:** it can provide a spatial context that makes it easier to understand the geographic distribution of the fields. This can be especially useful for people who are not familiar with the area or the crops being considered
* **Data exploration:** it can make it easier to explore the data contained in the JSON file. Users can zoom in and out, pan, and filter the data to focus on specific areas or types of crops
* **Data validation:** it can help validate the data contained in the JSON file. Users can visually confirm that the crop fields are located in the correct locations, and identify any potential errors or discrepancies in the data
* **Communication:** it can be a powerful way to communicate data to others. Users can share the map with stakeholders or the public to help them understand the geographic distribution of crop fields

In [ ]:
# Define the layer with a tooltip
layer = pdk.Layer(
    'PolygonLayer',
    data=fields_df,
    get_polygon='polygon_coordinates',
    get_fill_color=[255, 255, 0, 100],
    get_line_color=[255, 255, 0, 100],
    stroked=True,
    filled=True,
    lineWidthMinPixels=3,
    pickable=True,
    auto_highlight=True,
)

# Define the initial view state of the map
view_state = pdk.ViewState(
    longitude=fields_df.polygon_coordinates[0][0][0],
    latitude=fields_df.polygon_coordinates[0][0][1],
    zoom=12
)

# Create the map with the layers and the initial view state
r = pdk.Deck(layers=layer, initial_view_state=view_state,)

# Show the map
r

It can be noticed that our fields are placed in the Northern part of Spain. Please consider generalization issue.

## **Features extraction - using ee-satellites PyPI library**

The objective is to generate a dataset that contains for each field, for each time the satellites (sentinel 1, sentinel 2 and landsat 8) have passed on the field (in a period, specified by the user), all the phisical indicators that will be further used to build the final model.

This procedure has been designed to be performed in parallel in order to make multiple requests exploiting multi-threading.

Why you should use `ee-satellites`? [`Ee-satellites`](../README.md) is a PyPI library that provides **a useful, user friendly and powerful toolset for extracting satellite data in Python** (exploiting GEE APIs), and can be a valuable resource for researchers, analysts, and others working with Earth observation data.

### Select time-span to extract features

In [ ]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=datetime.date(2024, 4, 1), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date(2024, 4, 14), disabled=False)
display(end_date_widget)

### Sentinel 2 (optical features)

In [ ]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 2 satellites
fields_s2_features_extracted_df = ee_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, satellite='sentinel-2', filters_params=['40'], fields_threads=5)
# Add manure dates
display(fields_s2_features_extracted_df)

#### Store the obtained dataset

In [ ]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s2_features_extracted_df.to_csv(filename + '-s2-features-extracted.gz', header=True, index=False, compression='gzip')

### Sentinel 1 (radar features)

In [ ]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 1 satellites
fields_s1_features_extracted_df = ee_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, satellite='sentinel-1', filters_params=['ASCENDING'], fields_threads=5)
# Add manure dates
display(fields_s1_features_extracted_df)

#### Store the obtained dataset

In [ ]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s1_features_extracted_df.to_csv(filename + '-s1-features-extracted.gz', header=True, index=False, compression='gzip')

### Landsat 8 (optical + thermal features)

In [ ]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using landsat 8 satellites
fields_l8_features_extracted_df = ee_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, satellite='landsat-8', fields_threads=5)
# Add manure dates
display(fields_l8_features_extracted_df)

#### Store the obtained dataset

In [ ]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_l8_features_extracted_df.to_csv(filename + '-l8-features-extracted.gz', header=True, index=False, compression='gzip')